1. Installing the Dependencies

In [3]:
pip install nltk


  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/7e/d4/7ebdbd03970677812aac39c869717059dbb71a4cfc033ca6e5221787892c/click-8.1.8-py3-none-any.whl.metadata
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/38/ec/ad2d7de49a600cdb8dd78434a1aeffe28b9d6fc42eb36afab4a27ad23384/regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.5 kB 220.2 kB/s eta 0:00:01
     ---------------------------- --------- 30.7/41.5 kB 220.2 kB/s eta 0:00:01
     ----------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
# downloading the stopwords from nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Sadia
[nltk_data]     Tahsin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
#printing the stopwords
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

2. Data collection and pre-processing

In [14]:
#loading a dataset into a pandas dataframe
news_dataset = pd.read_csv(r"D:\OneDrive - GroupM Bangladesh\Machine Learning Projects\Siddhardhan\4. Fake News Prediction\train.csv")
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
#checking the number of rows and columns in the dataset
news_dataset.shape

(20800, 5)

In [16]:
#counting the number of missing values
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [17]:
#replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [19]:
#merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [21]:
#separating the data and label
X = news_dataset.drop(columns='label',axis=1)
y = news_dataset['label']

3. Stemming

It's a process to reducing a word to it's root word

In [22]:
port_stem = PorterStemmer()

In [23]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [24]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [25]:
#printing stemmed content
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [26]:
#separating the X and y again
X = news_dataset['content'].values
y = news_dataset['label'].values

In [27]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [28]:
print(y)

[1 0 1 ... 0 1 1]


In [48]:
#converting textual data to numerical data
vectorizer = TfidfTransformer()
vectorizer.fit(X)

X = vectorizer.transform(X)

print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210687 stored elements and shape (20800, 17128)>
  Coords	Values
  (0, 267)	0.052502702371268625
  (0, 2483)	0.6186771478715373
  (0, 2959)	0.025548403511459927
  (0, 3600)	0.5216329000022895
  (0, 3792)	0.05317825500823435
  (0, 4973)	0.016275692803685713
  (0, 7005)	0.009714521804451587
  (0, 7692)	0.026394355761491184
  (0, 8630)	0.09829250122643031
  (0, 8909)	0.5661419280098511
  (0, 13473)	0.03482428638009486
  (0, 15686)	0.08033530579150801
  (1, 1497)	0.07886484013832057
  (1, 1894)	0.0004762210596590218
  (1, 2223)	0.65071599678626
  (1, 2813)	0.002497541620426976
  (1, 3568)	0.03308342711683534
  (1, 5503)	0.7485378548097478
  (1, 6816)	0.002447783397302104
  (1, 16799)	0.09451528521484555
  (2, 2943)	0.02489594963259673
  (2, 3103)	0.4855719773861809
  (2, 5389)	0.11896165165721403
  (2, 5968)	0.05059263799398075
  (2, 9620)	0.8379879580176564
  :	:
  (20797, 3643)	0.010975196285210464
  (20797, 7042)	0.0139484582

4. Splitting the dataset to training and test data

In [41]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify = y)

5. Training the model: Logistic Regression Model

In [42]:
model = LogisticRegression()

In [43]:
model.fit(X_train,y_train)

LogisticRegression()

6. Evaluating the model

In [46]:
#accuracy score on training data
training_data_prediction = model.predict(X_train)
accuracy_1 = accuracy_score(training_data_prediction,y_train)

#accuracy score on test data
test_data_prediction = model.predict(X_test)

accuracy_2 = accuracy_score(test_data_prediction,y_test)

print(accuracy_1,accuracy_2)

0.98671875 0.9086538461538461


7. Building a predictive system

In [53]:
X_new = X_test[4]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
    print("The news is real")
else:
    print("The news is fake")

[1]
The news is fake
